In [23]:
# Import libraries
import cntk
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

# Training Set
# Vectors - [height, weight, shoe_size]
X = np.array([[181, 80, 44], [177, 70, 43], [160, 60, 38],
     [154, 54, 37], [166, 65, 40], [190, 90, 47],
     [175, 64, 39], [177, 70, 40], [159, 55, 37],
     [171, 75, 42], [181, 85, 43]])

# Labels - 1 per vector
Y = np.array([0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0])

# create model
model = Sequential()
model.add(Dense(60, input_dim=3, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# train model
model.fit(X, Y, epochs=100)

# save the model in ONNX format
cntk.combine(model.outputs).save("/mount/gender_classifier.onnx", format=cntk.ModelFormat.ONNX)

Epoch 1/100
11/11 [==============================] - 0s - loss: 0.6967 - acc: 0.4545
Epoch 2/100
11/11 [==============================] - 0s - loss: 0.6359 - acc: 0.9091
Epoch 3/100
11/11 [==============================] - 0s - loss: 0.6279 - acc: 0.5455
Epoch 4/100
11/11 [==============================] - 0s - loss: 0.6385 - acc: 0.5455
Epoch 5/100
11/11 [==============================] - 0s - loss: 0.6387 - acc: 0.5455
Epoch 6/100
11/11 [==============================] - 0s - loss: 0.6280 - acc: 0.5455
Epoch 7/100
11/11 [==============================] - 0s - loss: 0.6152 - acc: 0.5455
Epoch 8/100
11/11 [==============================] - 0s - loss: 0.6074 - acc: 0.5455
Epoch 9/100
11/11 [==============================] - 0s - loss: 0.6066 - acc: 0.8182
Epoch 10/100
11/11 [==============================] - 0s - loss: 0.6097 - acc: 0.8182
Epoch 11/100
11/11 [==============================] - 0s - loss: 0.6113 - acc: 0.8182
Epoch 12/100
11/11 [==============================] - 0s - loss

11/11 [==============================] - 0s - loss: 0.3788 - acc: 0.8182
Epoch 97/100
11/11 [==============================] - 0s - loss: 0.3762 - acc: 0.8182
Epoch 98/100
11/11 [==============================] - 0s - loss: 0.3736 - acc: 0.8182
Epoch 99/100
11/11 [==============================] - 0s - loss: 0.3711 - acc: 0.8182
Epoch 100/100
11/11 [==============================] - 0s - loss: 0.3686 - acc: 0.8182


TypeError: cannot convert list element to CNTK::Variable

In [6]:
!pip install --upgrade setuptools  --ignore-installed

  Using cached https://files.pythonhosted.org/packages/b2/86/095d2f7829badc207c893dd4ac767e871f6cd547145df797ea26baea4e2e/setuptools-41.2.0-py2.py3-none-any.whl


In [ ]:
# Test the ONNX model using the ONNX runtime
import onnxruntime as rt
import numpy as np

sess = rt.InferenceSession("/mount/gender_classifier.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
X_test = np.array([160,60,43])
pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]
print(pred_onx)


In [ ]:
# Store the model in MarkLogic

import requests
from requests.auth import HTTPDigestAuth

try:
    url = 'http://localhost:8011/v1/documents'
    header = {"Accept": "application/json"}
    params = 'uri=/tmp/gender_classifier.onnx'
    
    response = (requests.put(url+"?"+params, auth=HTTPDigestAuth('admin', 'admin'), headers=header, data=onx.SerializeToString()))
    print(response)
except Exception as e:
    raise e